1. Acquire data from the cloud database.

 You will want to end with a single dataframe. Include the logerror field and all other fields related to the properties that are available. You will end up using all the tables in the database.

Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid. - Only include properties with a transaction in 2017, and include only the last transaction for each property (so no duplicate property ID's), along with zestimate error and date of transaction. (Hint: read the docs for the .duplicated method) - Only include properties that have a latitude and longitude value.

2. Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

3. Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

In [1]:
import wrangle as w
import env as e
import pandas as pd

In [2]:
df=w.get_data()

In [3]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 52319 entries, 0 to 52318
Data columns (total 71 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   typeconstructiontypeid        76 non-null     float64
 1   storytypeid                   47 non-null     float64
 2   propertylandusetypeid         52319 non-null  float64
 3   heatingorsystemtypeid         33849 non-null  float64
 4   buildingclasstypeid           0 non-null      float64
 5   architecturalstyletypeid      70 non-null     float64
 6   airconditioningtypeid         13615 non-null  float64
 7   parcelid                      52319 non-null  int64  
 8   id                            52319 non-null  int64  
 9   basementsqft                  47 non-null     float64
 10  bathroomcnt                   52319 non-null  float64
 11  bedroomcnt                    52319 non-null  float64
 12  buildingqualitytypeid         33654 non-null  float64
 13  c

In [4]:
def missing_values(df):
    # Calculate the number of missing values for each attribute
    missing_counts = df.isnull().sum()
    
    # Calculate the percentage of missing values for each attribute
    total_rows = len(df)
    missing_percentages = (missing_counts / total_rows) * 100
    
    # Create a new DataFrame with the attribute name, missing count, and missing percentage
    result_df = pd.DataFrame({
        'Attribute': missing_counts.index,
        'Missing Count': missing_counts.values,
        'Missing Percentage': missing_percentages.values
    })
    
    return result_df


In [5]:
missing_values(df)

,Attribute,Missing Count,Missing Percentage
0,typeconstructiontypeid,52243,99.854737
1,storytypeid,52272,99.910166
2,propertylandusetypeid,0,0.000000
3,heatingorsystemtypeid,18470,35.302663
4,buildingclasstypeid,52319,100.000000
...,...,...,...
66,buildingclassdesc,52319,100.000000
67,heatingorsystemdesc,18470,35.302663
68,propertylandusedesc,0,0.000000
69,storydesc,52272,99.910166


Prepare:

1. Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer.

2. Create a function that will drop rows or columns based on the percent of values that are missing: handle_missing_values(df, prop_required_column, prop_required_row).

4. The input:
   A dataframe
   
   A number between 0 and 1 that represents the proportion, for each column, of rows with non-missing values required to keep the column. i.e. if prop_required_column = .6, then you are requiring a column to have at least 60% of values not-NA (no more than 40% missing).
   
   A number between 0 and 1 that represents the proportion, for each row, of columns/variables with non-missing values required to keep the row. For example, if prop_required_row = .75, then you are requiring a row to have at least 75% of variables with a non-missing value (no more that 25% missing).
   
- The output:
  The dataframe with the columns and rows dropped as indicated. Be sure to drop the columns prior to the rows in your function.
  
   hint:
2. Look up the dropna documentation.
3. You will want to compute a threshold from your input values (prop_required) and total number of rows or columns.
4. Encapsulate your work inside of functions in a wrangle_zillow.py module

In [6]:
df.head(3)

,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,parcelid,id,basementsqft,...,parcelid.1,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,NaN,NaN,261.0,NaN,NaN,NaN,NaN,14297519,1727539,NaN,...,14297519,0.025595,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
1,NaN,NaN,261.0,NaN,NaN,NaN,NaN,17052889,1387261,NaN,...,17052889,0.055619,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
2,NaN,NaN,261.0,NaN,NaN,NaN,NaN,14186244,11677,NaN,...,14186244,0.005383,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN


In [7]:
df.isnull().sum()

typeconstructiontypeid    52243
storytypeid               52272
propertylandusetypeid         0
heatingorsystemtypeid     18470
buildingclasstypeid       52319
                          ...  
buildingclassdesc         52319
heatingorsystemdesc       18470
propertylandusedesc           0
storydesc                 52272
typeconstructiondesc      52243
Length: 71, dtype: int64

In [8]:
def handle_missing_values(df, prop_required_column, prop_required_row):
    # Drop columns based on missing value percentage
    threshold_col = int(prop_required_column * len(df.index))
    df.dropna(axis=1, thresh=threshold_col, inplace=True)
    
    # Drop rows based on missing value percentage
    threshold_row = int(prop_required_row * len(df.columns))
    df.dropna(axis=0, thresh=threshold_row, inplace=True)
    
    return df

In [9]:
df=handle_missing_values(df, .97, .97)
df

,propertylandusetypeid,parcelid,id,bathroomcnt,bedroomcnt,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,...,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,max_transactiondate,id.1,parcelid.1,logerror,transactiondate,propertylandusedesc
0,261.0,14297519,1727539,3.5,4.0,3.5,3100.0,3100.0,6059.0,3.0,...,2016.0,537569.0,11013.72,6.059063e+13,2017-01-01,0,14297519,0.025595,2017-01-01,Single Family Residential
1,261.0,17052889,1387261,1.0,2.0,1.0,1465.0,1465.0,6111.0,1.0,...,2016.0,376000.0,5672.48,6.111001e+13,2017-01-01,1,17052889,0.055619,2017-01-01,Single Family Residential
2,261.0,14186244,11677,2.0,3.0,2.0,1243.0,1243.0,6059.0,2.0,...,2016.0,479489.0,6488.30,6.059022e+13,2017-01-01,2,14186244,0.005383,2017-01-01,Single Family Residential
3,261.0,12177905,2288172,3.0,4.0,3.0,2376.0,2376.0,6037.0,3.0,...,2016.0,36225.0,1777.51,6.037300e+13,2017-01-01,3,12177905,-0.103410,2017-01-01,Single Family Residential
4,261.0,12095076,781532,3.0,4.0,3.0,2962.0,2962.0,6037.0,3.0,...,2016.0,496619.0,9516.26,6.037461e+13,2017-01-01,6,12095076,-0.001011,2017-01-01,Single Family Residential
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52314,261.0,12412492,2274245,2.0,4.0,2.0,1633.0,1633.0,6037.0,2.0,...,2016.0,221068.0,4175.08,6.037555e+13,2017-09-19,77607,12412492,0.001082,2017-09-19,Single Family Residential
52315,261.0,11000655,673515,2.0,2.0,2.0,1286.0,1286.0,6037.0,2.0,...,2016.0,283704.0,4478.43,6.037101e+13,2017-09-20,77609,11000655,0.020615,2017-09-20,Single Family Residential
52316,261.0,17239384,2968375,2.0,4.0,2.0,1612.0,1612.0,6111.0,2.0,...,2016.0,16522.0,1107.48,6.111008e+13,2017-09-21,77610,17239384,0.013209,2017-09-21,Single Family Residential
52317,261.0,12773139,1843709,1.0,3.0,1.0,1032.0,1032.0,6037.0,1.0,...,2016.0,16749.0,876.43,6.037434e+13,2017-09-21,77611,12773139,0.037129,2017-09-21,Single Family Residential


In [10]:
df.isnull().sum()

propertylandusetypeid             0
parcelid                          0
id                                0
bathroomcnt                       0
bedroomcnt                        0
calculatedbathnbr                 0
calculatedfinishedsquarefeet      0
finishedsquarefeet12            139
fips                              0
fullbathcnt                       0
latitude                          0
longitude                         0
lotsizesquarefeet               277
propertycountylandusecode         0
rawcensustractandblock            0
regionidcity                    969
regionidcounty                    0
regionidzip                       1
roomcnt                           0
yearbuilt                        25
structuretaxvaluedollarcnt       57
taxvaluedollarcnt                 0
assessmentyear                    0
landtaxvaluedollarcnt             0
taxamount                         3
censustractandblock              45
max_transactiondate               0
id.1                        

In [11]:
missing_values(df)

,Attribute,Missing Count,Missing Percentage
0,propertylandusetypeid,0,0.000000
1,parcelid,0,0.000000
2,id,0,0.000000
3,bathroomcnt,0,0.000000
4,bedroomcnt,0,0.000000
5,calculatedbathnbr,0,0.000000
6,calculatedfinishedsquarefeet,0,0.000000
7,finishedsquarefeet12,139,0.266923
8,fips,0,0.000000
9,fullbathcnt,0,0.000000
